# Wstęp do multimediów (WMM)
### Kacper Górski, 331379


In [ ]:
from matplotlib import pyplot as plt
from IPython.display import display, Markdown
import cv2
import numpy as np
import copy
from wlasciwosci_obrazow import *
import os

Current working directory: c:\Users\Kacper\Desktop\WMM_25L\vision\image-properties
Image path: image-properties\monochrome\susie_mono.png


In [ ]:
NUM_IMAGES = 36
STUDENT_INDEX = 331379

def image_to_process(student_index, num_images):
    return student_index % num_images

print("Image index to process: ", image_to_process(STUDENT_INDEX, NUM_IMAGES))

 = os.path.join(os.getcwd(), "monochrome", "susie_mono.png")

image = cv2.imread(IMAGE_PATH, cv2.IMREAD_UNCHANGED)
print(image)
printi(image, "image")

Image index to process:  35
[[128 128 128 ... 128 128 128]
 [ 92  93  93 ...  16  16  16]
 [ 89  93  94 ...  16  16  16]
 ...
 [169 173 172 ...  16  16  16]
 [171 173 171 ...  16  16  16]
 [169 172 170 ...  16  16  16]]
image, wymiary: (480, 720), typ danych: uint8, wartości: 15 - 253


### Obraz do przetworzenia -> susie_col.png (index: 35)

## Dla obrazu monochromatycznego: 
➢ Obliczyć entropię obrazu wejściowego.   
➢ Wyznaczyć obraz różnicowy w taki sposób, że wartość każdego piksela zastępowana jest różnicą 
pomiędzy wartością tego piksela a wartością jego lewego sąsiada (kodowanie różnicowe /. 
predykcyjne). Dla pierwszego piksela w linii należy przyjąć wartość sąsiada równą 127 (środkowa 
wartość przedziału zmienności wartości pikseli).     
Wyświetlić obraz różnicowy.     
Wyznaczyć histogram obrazu różnicowego i porównać go z histogramem obrazu oryginalnego.   
Wyznaczyć entropię obrazu różnicowego i porównać ją z entropią obrazu oryginalnego.   
Uwaga: należy pamiętać, że wartości pikseli w obrazie różnicowym są z zakresu [-255; 255]!   
➢ Wyznaczyć współczynniki DWT korzystając z funkcji zamieszczonej w przykładowym skrypcie.  
Wyświetlić poszczególne pasma.   
Wyznaczyć histogramy i obliczyć entropię dla wszystkich pasm.   
Porównać wyniki (histogram, entropia) uzyskane dla poszczególnych pasm między sobą (czy któreś 
się wyróżniają i dlaczego?) oraz z wynikami uzyskanymi dla obrazu oryginalnego i obrazu 
różnicowego.    
Uwaga: należy pamiętać, że wartości w pasmach LH, HL i HH są z zakresu [-255, 255]!    
➢ Obliczyć przepływność (liczbę bitów przypadającą na jeden piksel) dla obrazu skompresowanego 
koderem PNG (obraz wejściowy) i porównać ją z obliczonymi uprzednio entropiami obrazu 
oryginalnego, różnicowego i falokowego. Czy przepływność mniejsza od entropii oznacza, że 
zależność: lśr ≥ H jest nieprawdziwa?